In [1]:
import json
import os
import re
import string

import pandas as pd
from datasets import Dataset, DatasetDict, load_from_disk

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
## LB에서 최고점이었던 코드만 남겨두고 나머지 경우는 주석 처리했습니다.
# 1. Datasets 불러오기
train_val = load_from_disk("./train_dataset/")
print(train_val)
test_ds = load_from_disk("./test_dataset/")
print(test_ds)

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 240
    })
})
DatasetDict({
    validation: Dataset({
        features: ['question', 'id'],
        num_rows: 600
    })
})


In [14]:
# 2. datasets를 DataFrame으로 변환
train = pd.DataFrame(train_val["train"])
dev = pd.DataFrame(train_val["validation"])
test = pd.DataFrame(test_ds["validation"])
# wiki = pd.read_json("wikipedia_documents.json") # 중첩 구조라 바로 안 읽힘.
with open("wikipedia_documents.json", "r", encoding="utf-8") as f:
    wiki_json = json.load(f)
values_list = list(wiki_json.values())
wiki = pd.DataFrame(values_list)

In [ ]:
"""
# 원본 csv 파일로 저장하기
train.to_csv('train.csv')
dev.to_csv('dev.csv')
test.to_csv('test.csv')
wiki.to_csv('wiki.csv')
"""

In [7]:
"""
# 3. wiki 'text' 중복 처리
# 3-1. wiki 'text' 중복 확인
print(wiki.shape)
wiki_drop = wiki.drop_duplicates(['text'])
print(wiki_drop.shape)
# 약 6.4%(3,876)의 중복 text 존재
"""

(60613, 8)
(56737, 8)


In [8]:
"""
# 3-2. wiki 'text' 중복 제거
unique_items = {}
for key, value in wiki_json.items():
    # 'text' 값이 unique_items에 없다면 추가
    if value['text'] not in unique_items:
        unique_items[value['text']] = value

# 중복이 제거된 항목을 중첩 구조로 다시 구성
# 키를 "0"부터 시작하는 숫자로 설정
reconstructed_json_data = {}
for index, (name, item) in enumerate(unique_items.items()):
    reconstructed_json_data[str(index)] = item

# wiki_json을 중복 제거된 내용으로 바꿔줌.
wiki_json = reconstructed_json_data

# 중복 제거 결과 확인
values_list = list(wiki_json.values())
wiki = pd.DataFrame(values_list)

print(wiki.shape)
print(wiki_drop.shape) # 아까 생성한 wiki_drop과 shape 일치 => 성공
"""

(56737, 8)
(56737, 8)


In [9]:
"""
# 4. 'question', 'answer'에 없는 'context'/'text' 내 특수 문자 처리
# 4-1. 특수 문자 찾기
special_characters = re.escape(string.punctuation)

def flatten_list(nested_list):
    for item in nested_list:
        if isinstance(item, list):
            yield from flatten_list(item)
        else:
            yield item

train['context_punc'] = [re.findall(f"[{special_characters}]", c) for c in train['context']]
train['question_punc'] = [re.findall(f"[{special_characters}]", c) for c in train['question']]
train['answers_punc'] = [re.findall(f"[{special_characters}]", c['text'][0]) for c in train['answers']]
train_c_punc = set(list(flatten_list(train['context_punc'].tolist())))
train_q_punc = set(list(flatten_list(train['question_punc'].tolist())))
train_a_punc = set(list(flatten_list(train['answers_punc'].tolist())))
print("train_c_punc: ", train_c_punc)
print("train_q_punc: ", train_q_punc)
print("train_a_punc: ", train_a_punc)

dev['context_punc'] = [re.findall(f"[{special_characters}]", c) for c in dev['context']]
dev['question_punc'] = [re.findall(f"[{special_characters}]", c) for c in dev['question']]
dev['answers_punc'] = [re.findall(f"[{special_characters}]", c['text'][0]) for c in dev['answers']]
dev_c_punc = set(list(flatten_list(dev['context_punc'].tolist())))
dev_q_punc = set(list(flatten_list(dev['question_punc'].tolist())))
dev_a_punc = set(list(flatten_list(dev['answers_punc'].tolist())))
print("dev_c_punc: ", dev_c_punc)
print("dev_q_punc: ", dev_q_punc)
print("dev_a_punc: ", dev_a_punc)

test['question_punc'] = [re.findall(f"[{special_characters}]", c) for c in test['question']]
test_q_punc = set(list(flatten_list(test['question_punc'].tolist())))
print("test_q_punc: ", test_q_punc)

wiki['text_punc'] = [re.findall(f"[{special_characters}]", c) for c in wiki['text']]
wiki_t_punc = set(list(flatten_list(wiki['text_punc'].tolist())))
print("wiki_t_punc: ", wiki_t_punc)

qa_punc = train_q_punc|dev_q_punc|test_q_punc|train_a_punc|dev_a_punc
print("train_c-qa_punc: ", train_c_punc - qa_punc)
print("dev_c-qa_punc: ", dev_c_punc - qa_punc)
print("wiki_t-qa_punc: ", wiki_t_punc - qa_punc)
"""

train_c_punc:  {':', '{', '`', '>', '*', '[', '?', '.', '\\', '/', '%', ')', '#', '!', '~', '&', '+', '|', ',', '"', '(', '}', "'", '_', '=', ']', '<', '$', '-', ';', '^'}
train_q_punc:  {'~', ':', '<', '/', '%', '-', ')', '>', ',', '"', '(', '?', '#', '.', "'", '!'}
train_a_punc:  {':', '~', '<', '/', '%', '+', '-', ')', ',', '>', '"', '(', '.', "'", '!'}
dev_c_punc:  {':', '>', '*', '[', '?', '.', '\\', '/', '%', ')', '#', '!', '~', '&', '+', '|', ',', '"', '(', "'", '=', ']', '<', '$', '-', ';'}
dev_q_punc:  {'~', '<', '%', ',', '>', '"', '?', '.', "'"}
dev_a_punc:  {':', ')', ',', '"', '(', '.', "'"}
test_q_punc:  {':', '<', '%', '-', ')', ',', '>', '"', '(', '?', '.', "'"}
wiki_t_punc:  {':', '{', '`', '>', '*', '[', '?', '.', '\\', '/', '%', ')', '#', '!', '~', '&', '+', '|', ',', '"', '(', '}', "'", '_', '=', ']', '<', '$', '-', ';', '^', '@'}
train_c-qa_punc:  {'_', '=', '&', ']', '$', '|', '{', '`', ';', '*', '^', '[', '}', '\\'}
dev_c-qa_punc:  {'=', '&', ']', '$', '|', ';', 

In [11]:
"""
# 4-2. 특수 문자 제거
# 삭제할 특수 문자들의 세트
special_chars = wiki_t_punc - qa_punc

# 특수 문자들을 정규 표현식 패턴으로 변환
# re.escape를 사용하여 특수 문자 앞에 자동으로 백슬래시를 추가
pattern = "[" + "".join(re.escape(char) for char in special_chars) + "]"
"""

In [ ]:
"""
# 4-3. 생성한 열 제거
train = train.drop(['context_punc', 'question_punc'], axis=1)
dev = dev.drop(['context_punc', 'question_punc'], axis=1)
"""

In [17]:
# 5. 일부 특수 문자('\n, '*', '#') and/or 특이한 리터럴('\\n') 처리
# wikipedia 문서에서 개행할 때('\n')나 불렛 포인트('*', '#')에 사용되는 문자는 불필요한 정보로 간주하고 삭제함. question, answer에도 부재.
# cf. '#'은 question에도 있지만(train_q_puc) 구체적인 사례를 살펴본 결과 단 1회였음. 소수 사례로 생각하고 일괄 처리함.

# 특수 문자를 제거하면 start index가 변경될 수 있기 때문에 start index를 변경하는 코드를 추가함.

# 5-1. start index 에러 검사
i = 0
for c, a in zip(train["context"], train["answers"]):
    st_idx = a["answer_start"][0]
    answer = a["text"][0]
    if answer != c[st_idx : st_idx + len(answer)]:
        i += 1

print("train st_idx error 횟수: ", i)

j = 0
for c, a in zip(dev["context"], dev["answers"]):
    st_idx = a["answer_start"][0]
    answer = a["text"][0]
    if answer != c[st_idx : st_idx + len(answer)]:
        j += 1

print("dev st_idx error 횟수: ", j)

## 원본 에러 없음.

train st_idx error 횟수:  0
dev st_idx error 횟수:  0


In [ ]:
"""
# 예시 문자열
# text = '오버워치오버워치#줄거리\n\n오버워치는 60년 이후 지구의 소설화된 미래를 배경으로 삼고 있다. 오버워치 단체는 이러한 미래로부터 30년 전에 설립되었다. 이러한 게임 이전의 사건은 솔저: 76 기원 이야기 비디오에서도 연대순으로 기록되어 있다 \n\n오버워치의 이야기는 "옴닉 사태"로부터 시작된다. 이 사건의 원인은 밝혀지지 않았다. 그러나 이 사건이 터지기 이전 인간은 지구의 경제적 평등을 창조하고 제조하기 위한 노력으로 인공지능인 옴닉을 개발했다. 이 인공지능 로봇들은 이후 옴니움으로 개발되었고, 이것은 그들을 만들기 위해 특별히 제작된 거대한 시설들이었다. 결국 세계의 옴니움들은 인간을 공격하는 적대적인 옴닉들을 만들기 시작했고, 옴닉 사태에 대응하기 위해 유엔은 군인과 과학자들로 구성된 오버워치라 불리는 기동대를 만들었다 게임에서 각각 리퍼와 솔저: 76로 알려진 가브리엘 레예스와 잭 모리슨이 오버워치 팀을 이끌었다. 모리슨의 전장에서의 성공이 레예스로부터 오버워치에 대한 지휘권을 인수받게 했고, 레예스는 오버워치의 비밀 작전 사단인 블랙워치를 이끌게 되었다. 옴닉 사태는 마침내 종결되었고 오버워치는 평화를 유지했다. 이 기간에 태어난 이들을 "오버워치 세대"라고 불렀다. 몇십 년 후, 오버워치는 부패, 미숙함, 무기 확산, 인권 남용을 비롯한 여러 논란에 휩쓸렸고, 이로 인해 오버워치에 대한 전세계적 시위가 일어났다 레예스와 모리슨 사이의 분쟁도 발생했으며, 유엔의 오버워치 시찰 동안 오버워치 본부에서 싸움이 발발해 폭발이 일어났고, 본부는 파괴되었으며 레예스와 모리슨 모두 죽은 것으로 여겨졌다. 유엔은 곧 오버워치의 이름으로 이뤄지는 모든 활동들을 불법으로 규정안 결의안을 통과시켰다. 페트라스 법안이라 불리는 이 결의안은 게임이 세팅되기 6년 전에 체결되었다 솔저: 76 기원 이야기에서 모리슨은 오버워치에 맞서는 논란들이 음모의 일부라고 여겼다 오버워치 단편 6개 중 5개와, 첫 시네마틱 트레일러는 이러한 뒷이야기 이후의 이야기들이다'
# answers = {'answer_start': [418], 'text': ['리퍼']}
# print(text[418:418+2])
# print(text.count('리퍼'))
# print(text.find('리퍼'))
"""

In [403]:
"""
# 5-2. 클리닝에 따른 start index 처리
# 5-2-1. 특수 문자('\n, '*', '#')와 리터럴('\\n') 공백 변환 후, 다중 공백을 단일 공백으로 수정한 경우 1, start index 처리
# 로컬 상에서는 가장 EM, f1이 높았으나(58.3333, 66.81530), LB에서는 원본 데이터보다 약간 하락함(34.17 -> 32.5, 47.41 -> 47.12).

def modify_st_idx(text, answers):
    # answer_start 업데이트
    original_answer_start = answers['answer_start'][0]
    updated_answer_start = original_answer_start

    # 첫 번째 클리닝에서 제거된 문자 수 계산
    before_first_cleaning = text[:original_answer_start]
    after_first_cleaning = re.sub("\\\\n", " ", before_first_cleaning)
    removed_chars_first_cleaning = len(before_first_cleaning) - len(after_first_cleaning)

    # 업데이트된 위치 계산
    updated_answer_start -= removed_chars_first_cleaning

    # 두 번째 클리닝에서 제거된 문자 수 계산 (첫 번째 클리닝 이후의 위치를 기준으로 함)
    before_second_cleaning = after_first_cleaning[:updated_answer_start]
    after_second_cleaning = re.sub(r"(\n|\*|\#)+", " ", before_second_cleaning)
    removed_chars_second_cleaning = len(before_second_cleaning) - len(after_second_cleaning)
    
    # 최종 업데이트된 위치 계산
    updated_answer_start -= removed_chars_second_cleaning
    
    # 세 번째 클리닝에서 제거된 문자 수 계산 (첫 번째 클리닝 이후의 위치를 기준으로 함)
    before_third_cleaning = after_second_cleaning[:updated_answer_start]
    after_third_cleaning = re.sub(r" +", " ", before_third_cleaning)
    removed_chars_third_cleaning = len(before_third_cleaning) - len(after_third_cleaning)

    # 최종 업데이트된 위치 계산
    updated_answer_start -= removed_chars_third_cleaning
    
    return updated_answer_start
"""

In [381]:
"""
# 5-2-2. 특수 문자('\n, '*', '#')와 리터럴('\\n') 공백 변환 후, 다중 공백을 단일 공백으로 수정한 경우 2, start index 처리
# 정규표현식 오류로 영문자 'n'이 삭제되며 st_idx error가 완전히 해결되지 않음.
# LB에서 F1 점수가 가장 높게 나옴(49.72). EM도 원본 데이터보다 약간 향상됨(34.17 -> 35.83).
def modify_st_idx(text, answers):
    # answer_start 업데이트
    original_answer_start = answers['answer_start'][0]
    updated_answer_start = original_answer_start

    # 첫 번째 클리닝에서 제거된 문자 수 계산
    before_first_cleaning = text[:original_answer_start]
    after_first_cleaning = re.sub("[\\\\n\n\#\*]", " ", before_first_cleaning)
    removed_chars_first_cleaning = len(before_first_cleaning) - len(after_first_cleaning)

    # 업데이트된 위치 계산
    updated_answer_start -= removed_chars_first_cleaning

    # 두 번째 클리닝에서 제거된 문자 수 계산 (첫 번째 클리닝 이후의 위치를 기준으로 함)
    before_second_cleaning = after_first_cleaning[:updated_answer_start]
    after_second_cleaning = re.sub(" +", " ", before_second_cleaning)
    removed_chars_second_cleaning = len(before_second_cleaning) - len(after_second_cleaning)
    
    # 최종 업데이트된 위치 계산
    updated_answer_start -= removed_chars_second_cleaning
    
    return updated_answer_start
"""

In [21]:
"""
# 5-2-3. 특수 문자('\n, '*', '#')만 공백 없이 제거한 경우, start index 처리
# 정규표현식 오류는 아닌데 '\\n'과 같은 리터럴이 새롭게 생성됨. 이유 불명.
# LB EM, F1 높음(36.25, 47,63). 그러나 이상하게 st_idx 처리 안 할 경우 더 높음;
def modify_st_idx(text, answers):
    # answer_start 업데이트
    original_answer_start = answers['answer_start'][0]
    updated_answer_start = original_answer_start

    # 첫 번째 클리닝에서 제거된 문자 수 계산
    before_first_cleaning = text[:original_answer_start]
    after_first_cleaning = re.sub("[\n\#\*]", "", before_first_cleaning)
    removed_chars_first_cleaning = len(before_first_cleaning) - len(after_first_cleaning)

    # 업데이트된 위치 계산
    updated_answer_start -= removed_chars_first_cleaning
    
    return updated_answer_start
"""

In [22]:
"""
# 5-3. st_idx 정정
for c, a in zip(train['context'], train['answers']):
    a['answer_start'] = [modify_st_idx(c, a)]

print(dev[218:219])
for c, a in zip(dev['context'], dev['answers']):
    a['answer_start'] = [modify_st_idx(c, a)]
print(dev[218:219])
"""

            title                                            context  \
218  오버워치 (애니메이션)  오버워치#줄거리\n\n오버워치는 60년 이후 지구의 소설화된 미래를 배경으로 삼고 ...   

                          question            id  \
218  블랙워치의 지도자가 게임에서 불리는 이름은 무엇인가?  mrc-1-000983   

                                     answers  document_id  __index_level_0__  
218  {'answer_start': [418], 'text': ['리퍼']}        53690                639  
            title                                            context  \
218  오버워치 (애니메이션)  오버워치#줄거리\n\n오버워치는 60년 이후 지구의 소설화된 미래를 배경으로 삼고 ...   

                          question            id  \
218  블랙워치의 지도자가 게임에서 불리는 이름은 무엇인가?  mrc-1-000983   

                                     answers  document_id  __index_level_0__  
218  {'answer_start': [417], 'text': ['리퍼']}        53690                639  


In [405]:
"""
5-4. 특수 문자 처리
5-4-1. 특수 문자('\n, '*', '#')와 리터럴('\\n') 공백 변환 후, 다중 공백을 단일 공백으로 수정한 경우 1
train['context'] = [re.sub(r"(\n|\*|\#)+", " ", c) for c in train['context']]
train['context'] = [re.sub("\\\\n", " ", c) for c in train['context']]
train['context'] = [re.sub(r" +", " ", c) for c in train['context']]

dev['context'] = [re.sub(r"(\n|\*|\#)+", " ", c) for c in dev['context']]
dev['context'] = [re.sub("\\\\n", " ", c) for c in dev['context']]
dev['context'] = [re.sub(r" +", " ", c) for c in dev['context']]
"""

In [ ]:
"""
5-4-2. 특수 문자('\n, '*', '#')와 리터럴('\\n') 공백 변환 후, 다중 공백을 단일 공백으로 수정한 경우 2
train['context'] = [re.sub("[\\\\n\n\#\*]", " ", c) for c in train['context']]
train['context'] = [re.sub(" +", " ", c) for c in train['context']]

dev['context'] = [re.sub("[\\\\n\n\#\*]", " ", c) for c in dev['context']]
dev['context'] = [re.sub(" +", " ", c) for c in dev['context']]
"""

In [24]:
# 5-4-3. 특수 문자('\n, '*', '#')만 공백 없이 제거한 경우
train["context"] = [re.sub("[\n\#\*]", "", c) for c in train["context"]]

dev["context"] = [re.sub("[\n\#\*]", "", c) for c in dev["context"]]

In [25]:
"""
# 5-5. start index 에러 검사
i = 0
for c, a in zip(train['context'], train['answers']):
    st_idx = a['answer_start'][0]
    answer = a['text'][0]
    if answer != c[st_idx:st_idx+len(answer)]:
        i += 1

        print(answer, c[st_idx:st_idx+len(answer)])

print('오류 횟수: ', i)

j = 0
for c, a in zip(dev['context'], dev['answers']):
    st_idx = a['answer_start'][0]
    answer = a['text'][0]
    if answer != c[st_idx:st_idx+len(answer)]:
        j += 1

        print(answer, c[st_idx:st_idx+len(answer)])

print('오류 횟수: ', j)
"""

오류 횟수:  0
오류 횟수:  0


In [29]:
dev["context"][218][:300]

'오버워치줄거리\\n\\n오버워치는 60년 이후 지구의 소설화된 미래를 배경으로 삼고 있다. 오버워치 단체는 이러한 미래로부터 30년 전에 설립되었다. 이러한 게임 이전의 사건은 솔저: 76 기원 이야기 비디오에서도 연대순으로 기록되어 있다 \\n\\n오버워치의 이야기는 "옴닉 사태"로부터 시작된다. 이 사건의 원인은 밝혀지지 않았다. 그러나 이 사건이 터지기 이전 인간은 지구의 경제적 평등을 창조하고 제조하기 위한 노력으로 인공지능인 옴닉을 개발했다. 이 인공지능 로봇들은 이후 옴니움으로 개발되었고, 이것은 그들을 만들기 위해 특별히 제작'

In [ ]:
# 5-6. wiki 특수 문자 처리
"""
# 5-6-1. 특수 문자('\n, '*', '#')와 리터럴('\\n') 공백 변환 후, 다중 공백을 단일 공백으로 수정한 경우 1
for key, value in wiki_json.items():
    value["text"] = re.sub(r"(\n|\*|\#)+", " ", value["text"])
    value["text"] = re.sub("\\\\n", " ", value["text"])
    value["text"] = re.sub(r" +", " ", value["text"])

# JSON 파일로 저장
json_file_path = "cleaned_wikipedia_documents.json"
with open(json_file_path, "w") as json_file:
    json.dump(wiki_json, json_file, ensure_ascii=False, indent=4)
"""

In [335]:
"""
# 5-6-2. 특수 문자('\n, '*', '#')와 리터럴('\\n') 공백 변환 후, 다중 공백을 단일 공백으로 수정한 경우 1
for key, value in wiki_json.items():
    value["text"] = re.sub("[\\\\n\n\#\*]", " ", value["text"])
    value["text"] = re.sub(" +", " ", value["text"])

# JSON 파일로 저장
json_file_path = "cleaned_wikipedia_documents.json"
with open(json_file_path, "w") as json_file:
    json.dump(wiki_json, json_file, ensure_ascii=False, indent=4)
"""

In [32]:
# 5-6-3. 특수 문자('\n, '*', '#')만 공백 없이 제거한 경우
for key, value in wiki_json.items():
    value["text"] = re.sub("[\n\#\*]", "", value["text"])

# JSON 파일로 저장
json_file_path = "cleaned_wikipedia_documents.json"
# json_file_path = "cleaned_dropped_wikipedia_documents.json" # 중복 제거한 경우
with open(json_file_path, "w") as json_file:
    json.dump(wiki_json, json_file, ensure_ascii=False, indent=4)

In [332]:
"""
# csv 파일로 저장
train.to_csv('cleaned_train.csv')
dev.to_csv('cleaned_dev.csv')
"""

In [31]:
# 5-7. DataFrame을 Dataset으로 저장
# 각 DataFrame을 다시 Dataset으로 변환
train_dataset = Dataset.from_pandas(train)
dev_dataset = Dataset.from_pandas(dev)

# 변환된 Dataset들을 DatasetDict으로 묶음
new_dataset_dict = DatasetDict({"train": train_dataset, "validation": dev_dataset})

# 변환된 DatasetDict을 디스크에 저장
new_dataset_dict.save_to_disk("./cleaned_train_dataset/")

Saving the dataset (1/1 shards): 100%|██████████| 240/240 [00:00<00:00, 15133.70 examples/s]
